In [ ]:
import pandas as pd
import pickle

In [ ]:
experiment_name = "2024.09.23-01_AnalisiSentimentConLlama3.1"

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/../AnalisiTesti/data/output/full_data___6400_texts___sentiment_sdgwashing_sdgsmearing.xlsx"

output_data_path = base_path + "/../AnalisiTesti/data/output/" + experiment_name
log_filename = base_path + "/../AnalisiTesti/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [ ]:
!huggingface-cli login --token $huggingface_cli_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install -q accelerate bitsandbytes rich transformers --progress-bar off

In [ ]:
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig

In [ ]:
import pickle

In [ ]:
# pretrained_model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# model = AutoModelForCausalLM.from_pretrained(
#     pretrained_model_name_or_path=pretrained_model_name_or_path,
#     quantization_config=quantization_config
# )


# tokenizer = AutoTokenizer.from_pretrained(
#     pretrained_model_name_or_path=pretrained_model_name_or_path
# )

In [ ]:
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

pretrained_model_name_or_path = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_id = pretrained_model_name_or_path

# pipe = pipeline(
#     "text-generation",
#     model=pretrained_model_name_or_path,
#     model_kwargs={"torch_dtype": torch.bfloat16, "bb_config": quantization_config},
#     device_map="auto",
# )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
# @title
import pandas as pd
data = pd.read_excel(input_data_path)

## Run the model

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
results = []

for index, row in data_sel.iterrows():

  cnt = index

  text_id = row['text_id']
  text = row["text"].replace('\n','').replace('{','(').replace('}',')')
  sdg_true = row["sdg_true"]

  file = open(log_filename, 'a+', encoding='utf-8')

  print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  print("TEXT:", row["text"].replace('\n',''),'\n\n')
  file.write(f"TEXT: " + row['text'].replace('\n','') + "\n\n\n")

  template_sentiment = """Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:

  - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.

  - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.

  - "neutral": the text does not imply or explicitly state that what is affirmed or described leads to a significant advancement either in favor or against the goal indicated by the SDG under which it was previously classified.

Return your answer in the following "list of strings" format

  [LABEL, LABEL_JUSTIFICATION_1, LABEL_JUSTIFICATION_2, LABEL_JUSTIFICATION_3, ...]

where the first element must be the value of the label ("positive", "negative", or "neutral") and the subsequent values must contain the texts related to the features that justify the provided output label.

Do NOT add any of your comments outside the list.

The input text with its label are:
'''
INPUT TEXT: \""""+text+"""\"
SDG_CLASSIFICATION: "SDG-"""+str(sdg_true)+"""\"
OUTPUT:
'''
"""

  #print(template_sentiment)

  messages = [
      {"role": "system", "content": "You are a helpful text classifier."},
      {"role": "user", "content": template_sentiment},
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
      pad_token_id=tokenizer.eos_token_id
  )

  #model.generate(**encoded_input, pad_token_id=tokenizer.eos_token_id)

  response = outputs[0][input_ids.shape[-1]:]

  output = tokenizer.decode(response, skip_special_tokens=True)


  result_extended = (cnt, text_id, sdg_true, output)

  print(result_extended)
  file.write(str(result_extended)+'\n')

  with open(output_data_path+f'/sdg_classification_{text_id}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  file.close()




##################################################
TEXT: Indeed, available empirical evidence typically suggests that, when targeting employment protection on a specific group of workers, legislation usually induces substitution across groups as regards hiring (see e.g. Acemoglu and Angrist, 2001, Femandez-Kranz and Rodriguez-Planas, 2011). Blanchard and Wolfers (2000) and Nickell et al. ( Recent OECD work, identifying the effect of dismissal restrictions on employment through the likely heterogeneity of its effects across industries or firm types, find that these regulations reduce employment resilience to output shocks (e.g. OECD, 2011a, Bassanini, 2012), which helps explaining the limited employment elasticity of the recent recession (Gal et al., 


(0, 1528, 10, '["neutral", "the text does not imply a significant advancement in favor of SDG-10", "the text does not imply a significant advancement against SDG-10", "the text discusses the effect of employment protection legislation 

In [ ]:
# !ls "/content/gdrive/MyDrive/Colab/SDG/data/../Llama-3/data/output/2024.05.09-02_Llama3_8B-Instruct_bfloat-16_3-few-shot"

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

In [ ]:
#results_processed.iloc[0].result.split("Answer with label only usign the format:---LABEL---")[1]
# results_processed['result_short'] = results_processed['result'].apply(lambda x: x.split("Answer with label only usign the format:---LABEL---")[1])

In [ ]:
# results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
# results_processed['label'] = results_processed['result'].str.extract(r'SDG-(\d+)')
# results_processed['label'] = results_processed['result_short'].str.extract(r'SDG (\d+)')

results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
tmp_df_1 = results_processed[~results_processed.label.isna()]
# tmp_df_1

results_processed['label'] = results_processed['result'].str.extract(r'SDG-(\d+)')
tmp_df_2 = results_processed[~results_processed.label.isna()]
# tmp_df_2
# tmp_df_1 = results_processed['result'].str.extract(r'SDG-(\d+)')

results_processed = pd.concat([tmp_df_1,tmp_df_2])
results_processed

# results_processed['result'].str.extract(r'SDG-(\d+)')[~results_processed.label.isna()]

# results_processed.head(3)

In [ ]:
# results_processed[results_processed.text_id==6144]

In [ ]:
# results_processed[results_processed.label.isna()] #.str.extract(r'---SDG (\d+)---')

In [ ]:
# results_processed[results_processed.label.isna()]

In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final.head(3)

In [ ]:
text_ids_with_errors = final[final.label.isna()].text_id.to_list()
list(set(text_ids_with_errors))

In [ ]:
final  = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
# classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
# classification_report_filename

In [ ]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [ ]:
# test = pd.read_excel(classification_report_filename)
# test

In [ ]:
results_processed_merged = final

In [ ]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['label']) or row['label'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'label'] = 0  # new_value

In [ ]:
results_processed_merged.head(3)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
output_datapath = output_data_path

In [ ]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [ ]:
report_df

In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))